In [72]:
# def parse_amazon_data(file_path):
# 	'''
# 	This function parses amazon data. It retrieves the reviews and good/bad label.

# 	:param file_path: path to traing/testing files
# 			-./data/test.ft.txt
# 			-./data/train.ft.txt
# 	:return: a tuple consisting of
# 			1. a list of reviews (a list of strings)
# 			2. a list of good/bad labels (stored as ints. 0 = bad, 1 = good)
# 	'''
# 	stopwords_set = set(stopwords.words('english'))
# 	lemmatizer = WordNetLemmatizer().lemmatize

# 	review_str_list = []
# 	labels_list = []

# 	with open(file_path, 'r') as f:
# 		words = []
# 		for line in f:
# 			label_str, review_str = line.split(' ', 1)
# 			label = int(label_str[-1]) - 1		# original data is labeled with 1 and 2. change to labels of 0 and 1
# 			# words = [lemmatizer(w) for w in review_str.lower().split() if w not in stopwords_set]	# get the words out
# 			review_str_list.append(review_str_list)
		
# 			# Not too sure what the best DS is here but gonna store everything in list for now
# 			# review_str_list.append(words)
# 			labels_list.append(label)

# 	return [review_str_list, labels_list]

In [50]:
training_amazon = "./data/train.ft.txt"
testing_amazon = "./data/test.ft.txt"
training_imdb = "./data/aclImdb/train"
testing_imdb = "./data/aclImdb/test"

In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import bz2
import os
print(os.listdir("./data"))

['.DS_Store', 'aclImdb', 'train.ft.txt', 'test.ft.txt.bz2', 'test.ft.txt', 'train.ft.txt.bz2', 'trainingandtestdata']


In [60]:
trainfile = bz2.BZ2File('./data/test.ft.txt.bz2','r')
lines = trainfile.readlines()

In [63]:
lines[3]

b"__label__2 works fine, but Maha Energy is better: Check out Maha Energy's website. Their Powerex MH-C204F charger works in 100 minutes for rapid charge, with option for slower charge (better for batteries). And they have 2200 mAh batteries.\n"

In [91]:
def parse_amazon_data(lines):
    review_and_ratings_list=[]
    for i in range(len(lines)):
        line_text = str(lines[i])
        label, review_str = line_text.split(" ", 1)
        label = int(label[-1]) - 1         # label with 1 and 0
        review_and_ratings_list.append([review_str, label])
    return review_and_ratings_list

In [96]:
review_and_ratings_list=parse_amazon_data(lines)

In [98]:
review_and_ratings_list[0]

['Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\\\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\\\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"\\n\'',
 1]

In [101]:
train_df = pd.DataFrame(review_and_ratings_list,columns=['review','rating'])
train_df.head()

,review,rating
0,Great CD: My lovely Pat has one of the GREAT v...,1
1,One of the best game music soundtracks - for a...,1
2,Batteries died within a year ...: I bought thi...,0
3,"works fine, but Maha Energy is better: Check o...",1
4,Great for the non-audiophile: Reviewed quite a...,1


In [102]:
train_df['rating'].value_counts()

1    200000
0    200000
Name: rating, dtype: int64

In [104]:
train_df['word_count'] = train_df['review'].str.lower().str.split().apply(len)
train_df.head()

,review,rating,word_count
0,Great CD: My lovely Pat has one of the GREAT v...,1,106
1,One of the best game music soundtracks - for a...,1,148
2,Batteries died within a year ...: I bought thi...,0,60
3,"works fine, but Maha Energy is better: Check o...",1,37
4,Great for the non-audiophile: Reviewed quite a...,1,69


In [105]:
import string 
def remove_punc(s):
    table = str.maketrans({key: None for key in string.punctuation})
    return s.translate(table)

In [107]:
train_df['review'] = train_df['review'].apply(remove_punc)
train_df.shape

(400000, 3)

In [108]:
train_df.head()

,review,rating,word_count
0,Great CD My lovely Pat has one of the GREAT vo...,1,106
1,One of the best game music soundtracks for a ...,1,148
2,Batteries died within a year I bought this ch...,0,60
3,works fine but Maha Energy is better Check out...,1,37
4,Great for the nonaudiophile Reviewed quite a b...,1,69


In [111]:
len(train_df['word_count'][train_df['word_count']<=25])

25768

In [112]:
train_df1 = train_df[:][train_df['word_count']<=25]
train_df1.head()

,review,rating,word_count
20,Long and boring Ive read this book with much e...,0,19
44,Amazing love itvery touchits to bad that in th...,1,15
55,No another grill like this This grill is one o...,1,24
88,Creepy The creepiest book Ive ever read Its a ...,1,22
99,Possibly the funniest movie ever made It start...,1,22


In [113]:
train_df1['rating'].value_counts()

1    15790
0     9978
Name: rating, dtype: int64

In [115]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
st_wd = text.ENGLISH_STOP_WORDS
c_vector = CountVectorizer(stop_words = st_wd,min_df=.0001,lowercase=1)
X_counts = c_vector.fit_transform(train_df1['review'].values)

In [116]:
X_counts

<25768x9678 sparse matrix of type '<class 'numpy.int64'>'
	with 243841 stored elements in Compressed Sparse Row format>

In [123]:
from sklearn.model_selection import train_test_split
y = train_df1['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size=0.1, random_state=42)

In [124]:
X_train = X_train.todense()
X_test = X_test.todense()

In [125]:
X_train.shape

(23191, 9678)

In [126]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model1=  Sequential()
model1.add(Dense(1000,input_shape=(9678,),activation='relu'))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
hist = model1.fit(X_train,y_train,epochs=6,batch_size=128,verbose=1)

Epoch 1/6
23191/23191 [==============================] - 36s 2ms/step - loss: 0.3239 - acc: 0.8577: 8s - loss: 0.3370 - acc: 0. 
Epoch 2/6
23191/23191 [==============================] - 39s 2ms/step - loss: 0.1575 - acc: 0.9398
Epoch 3/6
23191/23191 [==============================] - 34s 1ms/step - loss: 0.0847 - acc: 0.9729
Epoch 4/6
23191/23191 [==============================] - 32s 1ms/step - loss: 0.0384 - acc: 0.9901
Epoch 5/6
23191/23191 [==============================] - 34s 1ms/step - loss: 0.0154 - acc: 0.9974
Epoch 6/6
23191/23191 [==============================] - 33s 1ms/step - loss: 0.0066 - acc: 0.9996


In [127]:
model1.evaluate(X_test, y_test, batch_size=128)

2577/2577 [==============================] - 2s 641us/step


[0.5244230652646497, 0.8820333721612746]

In [128]:
model1.evaluate(X_train, y_train, batch_size=128)

23191/23191 [==============================] - 9s 400us/step


[0.0032562388347812084, 0.999870639472209]